<h1>Содержание<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Подготовка" data-toc-modified-id="Подготовка-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Подготовка</a></span></li><li><span><a href="#Обучение" data-toc-modified-id="Обучение-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Обучение</a></span></li><li><span><a href="#Выводы" data-toc-modified-id="Выводы-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Выводы</a></span></li></ul></div>

# Проект для «Викишоп» c BERT

Интернет-магазин «Викишоп» запускает новый сервис. Теперь пользователи могут редактировать и дополнять описания товаров, как в вики-сообществах. То есть клиенты предлагают свои правки и комментируют изменения других. Магазину нужен инструмент, который будет искать токсичные комментарии и отправлять их на модерацию. 

Обучите модель классифицировать комментарии на позитивные и негативные. В вашем распоряжении набор данных с разметкой о токсичности правок.

Постройте модель со значением метрики качества *F1* не меньше 0.75. 

**Инструкция по выполнению проекта**

1. Загрузите и подготовьте данные.
2. Обучите разные модели. 
3. Сделайте выводы.

Для выполнения проекта применять *BERT* необязательно, но вы можете попробовать.

**Описание данных**

Данные находятся в файле `toxic_comments.csv`. Столбец *text* в нём содержит текст комментария, а *toxic* — целевой признак.

## Подготовка

In [1]:
import warnings
warnings.filterwarnings('ignore')

import numpy as np
import pandas as pd
import torch
import transformers
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import SGDClassifier
from sklearn.ensemble import VotingClassifier
from lightgbm import LGBMClassifier
from catboost import CatBoostClassifier
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import train_test_split
from tqdm import notebook
from sklearn.metrics import f1_score
from sklearn.utils import shuffle
from transformers import logging
logging.set_verbosity_warning()

df = pd.read_csv('https://code.s3.yandex.net/datasets/toxic_comments.csv')

In [2]:
print(df.shape)

Перебросим расчеты эмбеддингов на GPU (при наличии ядер CUDA). Тестировал на своей машине - ускорение примерно в 5 раз.

In [3]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

cuda:0


Оценим дизбаланс классов.

In [4]:
print(df['toxic'].value_counts(normalize = True))

0    0.898321
1    0.101679
Name: toxic, dtype: float64


Дизбаланс классов впечатляющий. Учитывая скорость расчета эмбеддингов, не будем делать downsampling для балансировки классов, на практике для этого датасета он показал себя хуже, чем гиперпараметр class_weight = 'balanced' у моделей.

Сделаем семпл, близкий к исходному датафрейму, но содержащий кол-во элементов, кратное 50. 50 - размер батча. Кратное 50 нужно сделать, чтобы впоследствии фичи и таргеты не расходились в количестве (неполный батч эмбеддиться не будет)

In [5]:
#features = np.concatenate(embeddings)
#df_sample = df[df['text'].str.len() <= 512].sample(8000).reset_index(drop = True)
#df_sample = df[:100000]
df = df.sample(159500).reset_index(drop = True)
features = df['text']
target = df['toxic']


Для ускорения расчетов используем DistilBert, тем более, что обычный Bert на данном датафрейме показал себя хуже (что парадоксально)

In [6]:
tokenizer = transformers.DistilBertTokenizer.from_pretrained('distilbert-base-uncased')
#tokenizer = transformers.BertTokenizer.from_pretrained('bert-base-uncased')

tokenized = features.apply( #sample['text'].apply(
    lambda x: tokenizer.encode(x, add_special_tokens=True, max_length = tokenizer.model_max_length,truncation=True ))

max_len = 0
for i in tokenized.values:
    if len(i) > max_len:
        max_len = len(i)

padded = np.array([i + [0]*(max_len - len(i)) for i in tokenized.values])

attention_mask = np.where(padded != 0, 1, 0)

In [7]:
print(attention_mask.shape)

(159500, 512)


In [8]:
#model = transformers.DistilBertModel.from_pretrained('distilbert-base-uncased')
model = transformers.DistilBertModel.from_pretrained('distilbert-base-uncased')
#model = transformers.BertModel.from_pretrained('bert-base-uncased')

Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertModel: ['vocab_transform.weight', 'vocab_projector.bias', 'vocab_layer_norm.weight', 'vocab_layer_norm.bias', 'vocab_transform.bias', 'vocab_projector.weight']
- This IS expected if you are initializing DistilBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [9]:
batch_size = 100
embeddings = []

for i in notebook.tqdm(range(padded.shape[0] // batch_size)):
        batch = torch.cuda.IntTensor(padded[batch_size*i:batch_size*(i+1)], device = device) 
        attention_mask_batch = torch.cuda.IntTensor(attention_mask[batch_size*i:batch_size*(i+1)], device = device)
        
        with torch.no_grad():
            model.to(device)
            batch_embeddings = model(batch, attention_mask=attention_mask_batch)
        
        embeddings.append (torch.Tensor.cpu(batch_embeddings[0][:,0,:]).numpy())#(batch_embeddings[0][:,0,:]).numpy()


  0%|          | 0/1595 [00:00<?, ?it/s]

Проверим количество получившихся эмбеддингов для тренировочной выборки.

In [10]:
#embed = torch.Tensor.cpu(embeddings)
print(np.concatenate(embeddings).shape)

(159500, 768)


## Обучение

In [11]:
features = np.concatenate(embeddings)
#features_test = np.concatenate(embeddings_test)

Разобьем таргет и получившиеся эмбеддинги на тестовую и тренировочную выборку в соотношении 20/80.

In [12]:
features_train, features_test, target_train, target_test = train_test_split(features, 
                                                                            target, test_size = 0.2, random_state = 42)

Проверим корректность сплита

In [13]:
print(features_test.shape)

(31900, 768)


В качестве моделей используем Логистическую регрессию, LGBMClassifier, CatBoostClassifier, RandomForest Classifier, RidgeClassifier, VotingClassifier, SGDClassifier.

In [14]:

model = LogisticRegression(class_weight = 'balanced', random_state = 12345)
model.fit(features_train, target_train)
predictions = model.predict(features_test)
preds_train = model.predict(features_train)
score = f1_score(predictions, target_test)
print(score)

0.6630397036694062


In [15]:
from sklearn.metrics import accuracy_score
model_lgbm = LGBMClassifier(class_weight = 'balanced', random_state=42, num_iterations = 300)# n_estimators=80, max_depth = 9, num_iterations = 500)
model_lgbm.fit(features_train, target_train)
predicted_test = model_lgbm.predict(features_test)
score_lgbm = f1_score(predicted_test, target_test)
print(score_lgbm)

0.7047259997307123


In [16]:
model_cb = CatBoostClassifier( random_state=42, eval_metric = 'F1', 
                              iterations = 1000, auto_class_weights = 'Balanced')# , depth=9)
model_cb.fit(features_train, target_train, verbose=100)
predicted_cb = model_cb.predict(features_test)
score_cb = f1_score(predicted_cb, target_test)
print(score_cb)

Learning rate set to 0.081683
0:	learn: 0.7884850	total: 720ms	remaining: 11m 59s
100:	learn: 0.8907708	total: 16.1s	remaining: 2m 23s
200:	learn: 0.9100554	total: 30.5s	remaining: 2m 1s
300:	learn: 0.9252283	total: 44.9s	remaining: 1m 44s
400:	learn: 0.9392663	total: 59s	remaining: 1m 28s
500:	learn: 0.9506036	total: 1m 12s	remaining: 1m 12s
600:	learn: 0.9603182	total: 1m 27s	remaining: 58.1s
700:	learn: 0.9674765	total: 1m 41s	remaining: 43.4s
800:	learn: 0.9727545	total: 1m 55s	remaining: 28.7s
900:	learn: 0.9765909	total: 2m 9s	remaining: 14.2s
999:	learn: 0.9794623	total: 2m 23s	remaining: 0us
0.7058980874682359


In [17]:
from sklearn.ensemble import RandomForestClassifier
model_forest = RandomForestClassifier(random_state=42, n_estimators=120, max_depth = 8)
model_forest.fit(features_train, target_train)
predicted_forest = model_forest.predict(features_test)
score_cb = f1_score(predicted_forest, target_test)
print(score_cb)

0.5286481647269472


In [19]:
from sklearn.linear_model import RidgeClassifier
model_rdg = RidgeClassifier(class_weight = 'balanced')
model_rdg.fit(features_train, target_train)
predictions = model_rdg.predict(features_test)
score = f1_score(predictions, target_test)
print(score)

0.64889997731912


In [20]:

m1 = LGBMClassifier(class_weight = 'balanced', random_state=42, num_iterations = 400)
m2 = SGDClassifier(class_weight = 'balanced', random_state = 12345)
m3 = LogisticRegression(class_weight = 'balanced')
m4 = RandomForestClassifier(random_state=42, n_estimators=80, max_depth = 9)
model_vtng = VotingClassifier(estimators=[('lgbm', m1), ('sgdc', m2), ('regr', m3)], voting='hard')
model_vtng.fit(features_train, target_train)
predictions = model_vtng.predict(features_test)
score = f1_score(predictions, target_test)
print(score)

0.6618257261410788


In [24]:

model_sgd = SGDClassifier(class_weight = 'balanced')#, random_state = 42)
model_sgd.fit(features_train, target_train)
predictions = model_sgd.predict(features_test)
score = f1_score(predictions, target_test)
print(score)

0.751412429378531


## Выводы

В целом, BERT на данном датасете показала себя достаточно плохо. Были предприняты попытки взятия разных семплов (2000, 10000, 20000, полный датасет, те записи, где длина меньше 512 символов), использование разных предтренированных моделей, но в среднем F1 на данных настройках при применении любых моделей не превышает 0.7. При обработке полной выборки лучше всего показал себя SGDClassifier c результатом 0.75 (надо заметить, не без рандома).